In [37]:
Plot = require 'itorch.Plot';

In [38]:
require 'cutorch'
require 'cunn'

require 'nn';
require 'model';
require 'mnist_loader';

cutorch.setDevice(2)


model = nn.Sequential()
model:add(nn.Reshape(32*32))
model:add(nn.Linear(32*32, 100))
model:add(nn.ReLU())
model:add(nn.Linear(100, 10))
model:add(nn.ReLU())
model:add(nn.LogSoftMax())

criterion = nn.ClassNLLCriterion()

model:cuda()
criterion:cuda()

In [39]:
train_set = mnist.loadTrainSet()
val_set = mnist.loadTestSet()

<mnist> done	


<mnist> done	


In [40]:
function get_label(one_hot) 
    local label = -1
    for i=1, one_hot:size()[1] do
        if one_hot[i] == 1 then
            label = i
        end
    end
    return label
end
data = train_set[1][1]:cuda()
label = get_label(train_set[1][2])

output = model:forward(data)
print(output)
cost = criterion:forward(output, label)
print(cost)

-2.3812 -2.2850 -2.1748 -2.1542 -2.1719 -2.3812 -2.3812 -2.3812 -2.3812 -2.3812
[torch.CudaTensor of size 1x10]



In [41]:
learningrate = 1e-2
batchsize = 100
start_batchsize = 1
end_batchsize = train_set:size()/batchsize
cost_tb = {}
validation_tb = {}
batch_idx_tb = {}
parameters,gradParameters = model:getParameters()
have_print = false
epochsize = 5
counter = 1
for epoch = 1, epochsize do
for i = 1,  end_batchsize do
    local total_cost = 0
    
    local batch_data = torch.zeros(batchsize, 32, 32):cuda()
    local batch_label = {}
    
    for idx = 1, batchsize do
        data_idx = (i - 1) * batchsize + idx
        vec = train_set[data_idx][1]:cuda()
        label = get_label(train_set[data_idx][2])
        
        batch_data[{{idx},}] = vec
        table.insert(batch_label, label)
    end
    

    output = model:forward(batch_data)
    

    total_cost = 0
    
    
    back_grad = torch.zeros(batchsize, 10):cuda()
    for idx = 1, batchsize do
        cost = criterion:forward(output[idx], batch_label[idx])
        total_cost = total_cost + cost
        
        c_b = criterion:backward(output[idx], batch_label[idx])
        back_grad[{{idx}, }] = c_b    
    end
    model:zeroGradParameters()
    model:backward(batch_data, back_grad)
    gradParameters:div(batchsize)
    
    
    model:updateParameters(learningrate)
    
    total_cost = total_cost / batchsize        
        
    if not have_print then
        par, grad_par = model:parameters()
        print(model.gradInput:size())
        have_print = true
    end
    
    local validation_total_cost = 0
    selective_val_count = batchsize
    for j = 1, selective_val_count do
        vec = val_set[j][1]:cuda()
        label = get_label(val_set[j][2])
        output = model:forward(vec)
        cost = criterion:forward(output, label)
        validation_total_cost = validation_total_cost + cost
    end
    validation_total_cost = validation_total_cost / selective_val_count    
    
    table.insert(cost_tb, total_cost)
    table.insert(validation_tb, validation_total_cost)
    table.insert(batch_idx_tb, counter)
    counter = counter + 1
end
end

 100
  32
  32
[torch.LongStorage of size 3]



In [42]:
plot = Plot():line(batch_idx_tb, cost_tb, 'red', 'traning')
plot:line(batch_idx_tb, validation_tb, 'blue', 'validation')
plot:title('Scatter Plot Demo')
plot:xaxis('x'):yaxis('y')
plot:legend(true):draw()
plot:save('mini_batch.html')